In [ ]:
import scanpy as sc
import squidpy as sq

from Algorithm.distribution import *
from util import *

In [ ]:
ctrl_A = anndata.read_h5ad('E://human/10X_Visium_hunter2021spatially_sample_A_data.h5ad')
adata_A = anndata.read_h5ad('E://human/10X_Visium_hunter2021spatially_sample_A_data.h5ad')
adata_B = anndata.read_h5ad('E://human/10X_Visium_hunter2021spatially_sample_B_data.h5ad')
adata_C = anndata.read_h5ad('E://human/10X_Visium_hunter2021spatially_sample_C_data.h5ad')

In [ ]:
h5_path = 'E://human/GSM4838131_Visium_Sample_A_filtered_feature_bc_matrix.h5'
csv_path = 'E://human/GSM4838131_Visium_Sample_A_tissue_positions_list.csv'

h5_path = 'E://human/GSM4838133_Visium_Sample_C_filtered_feature_bc_matrix.h5'
csv_path = 'E://human/GSM4838133_Visium_Sample_C_tissue_positions_list.csv'

h5_path = '/Users/sunpeisen/Downloads/data/GSM4838132_Visium_Sample_B_filtered_feature_bc_matrix.h5'
csv_path = '/Users/sunpeisen/Downloads/data/GSM4838132_Visium_Sample_B_tissue_positions_list.csv'

h5_path = 'E://human/GSM4838132_Visium_Sample_B_filtered_feature_bc_matrix.h5'
csv_path = 'E://human/GSM4838132_Visium_Sample_B_tissue_positions_list.csv'

adata = sc.read_10x_h5(h5_path)
add_spatial_position(adata, csv_path)
adata.var_names_make_unique()
sc.pp.calculate_qc_metrics(adata, inplace=True)

#sc.pp.filter_genes(adata, min_cells=2)
adata.var['mt'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
adata.var_names_make_unique()

#adata = adata[adata.obs["pct_counts_mt"] < 8]
sc.pl.violin(adata, 
             ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)


In [ ]:
sc.pp.filter_genes(adata, min_cells=500)
adata

In [ ]:
result = convolve(get_3d_matrix(adata), kernel=get_gaussian_kernel(size=5))
update_anndata(result, adata)

In [ ]:
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pl.highly_variable_genes(adata)

In [ ]:
gene_list = adata.var[adata.var['highly_variable']==True].index

In [ ]:
len(gene_list)

In [ ]:
from Algorithm.graph import *
gmm_dict = fit_gmms(adata, gene_list, n_comp=20)

In [ ]:
def build_graph(gmm_dict: dict, distance_threshold: int = 1):
    # build graph
    graph = nx.Graph()
    gene_list = list(gmm_dict.keys())
    gene_counts = len(gene_list)
    # add nodes
    for gene_id in gene_list:
        graph.add_node(gene_id)
    # calculate the weight and add edges
    for i in range(gene_counts):
        for j in range(gene_counts):
            if i != j and not graph.has_edge(gene_list[i], gene_list[j]):
                distance = distribution_distance(gmm_dict[gene_list[i]], gmm_dict[gene_list[j]])
                if distance < 0:
                    print(distance)
                    print(gene_list[i]+'...'+gene_list[j])
                if distance < distance_threshold:
                    weight = 1 / distance
                    graph.add_edge(gene_list[i], gene_list[j], weight=weight)
    return graph


g = build_graph(gmm_dict, distance_threshold=5)
g = modify_graph(g)

In [ ]:
sc.pl.spatial(adata,
              size=2,
              spot_size=300,
              color=list(['actn3b','pdlim5a','ppp1r3ab','si:ch211-266g18.10']))

In [48]:
def get_hellinger_distance(gmm1_covs, gmm1_means, gmm2_covs, gmm2_means):
    mean_cov = (gmm1_covs + gmm2_covs) / 2
    mean_cov_det = np.linalg.det(mean_cov)
    mean_cov_inv = np.linalg.inv(mean_cov)
    gmm1_cov_det = np.linalg.det(gmm1_covs)
    gmm2_cov_det = np.linalg.det(gmm2_covs)
    means_diff = gmm1_means - gmm2_means
    first_term = np.exp(-(means_diff.T @ mean_cov_inv @ means_diff) / 8)
    second_term = ((np.power(gmm1_cov_det, 1 / 4)) * np.power(gmm2_cov_det, (1 / 4))) / np.sqrt(mean_cov_det)
    hellinger_distance = np.sqrt(np.abs(1 - first_term * second_term))
    return hellinger_distance

def distribution_distance(gmm1, gmm2):
    gmm1_weights = gmm1.weights_
    gmm1_means = gmm1.means_
    gmm1_covs = gmm1.covariances_
    gmm2_weights = gmm2.weights_
    gmm2_means = gmm2.means_
    gmm2_covs = gmm2.covariances_
    n_components = gmm1_weights.size
    # calculate the distance
    distance_array = np.zeros((n_components, n_components))
    for i in range(n_components):
        for j in range(n_components):
            distance_array[i, j] = get_hellinger_distance(gmm1_covs[i], gmm1_means[i], gmm2_covs[j], gmm2_means[j])
    # TODO: consider the weight of each component
    min_cost = linear_sum(distance_array)
    return min_cost


distribution_distance(gmm_dict['actn3b'], gmm_dict['si:ch211-266g18.10'])

3.28686177892122

In [ ]:
gmm_dict['actn3b']


In [ ]:
plt.figure(figsize=(8, 8))
pos = nx.spring_layout(g)

nx.draw_networkx(g, pos=pos, with_labels=True, node_size=600, width=2, node_color='yellow')
plt.show()

In [ ]:
from sklearn.cluster import SpectralClustering

similarity_matrix  = nx.to_numpy_array(g)
k = 2  # 聚类数

clustering_model = SpectralClustering(n_clusters=k, affinity='precomputed', n_init=100)
clustering_model.fit(similarity_matrix)

# 打印聚类结果
print("Cluster labels:", clustering_model.labels_)

In [ ]:
similarity_matrix

In [ ]:
clustering_model.labels_

In [ ]:
similarity_matrix

In [ ]:
list(nx.spring_layout(g).keys())

In [ ]:
similarity_matrix

In [ ]:
import networkx as nx

# 使用Louvain算法进行聚类
comp = nx.algorithms.community.girvan_newman(g)

# 打印结果
for i, c in enumerate(comp):
    print(f"Community {i+1}: {c}")

In [ ]:
import networkx as nx
adj_matrix = nx.to_numpy_matrix(g)

In [ ]:
degree_matrix = np.diag(np.sum(adj_matrix, axis=1))
laplacian_matrix = degree_matrix - adj_matrix

eigenvalues, eigenvectors = np.linalg.eig(np.asarray(laplacian_matrix))

k = 10
U = eigenvectors[:, 0:k]

normalized_U = U / np.linalg.norm(U, axis=1).reshape(-1, 1)

from sklearn.cluster import KMeans

kmeans = KMeans()
kmeans.fit(normalized_U)


plt.scatter(np.array(U[:,0]), np.array(U[:,1]), c=kmeans.labels_)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms.community import greedy_modularity_communities

# 使用贪心算法计算模块化谱图
communities = list(greedy_modularity_communities(g))

# 可视化模块化谱图
pos = nx.spring_layout(g)
for i, community in enumerate(communities):
    nx.draw_networkx_nodes(g, pos, nodelist=list(community), node_color=f'C{i}', node_size=100, alpha=0.8)
nx.draw_networkx_edges(g, pos, width=1, alpha=0.5)
plt.show()

In [ ]:
# TODO: know why
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata, max_value=10)

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors=30, n_pcs=50)
sc.tl.umap(adata)
sc.tl.leiden(adata)
sc.pl.umap(adata, color='leiden')

In [ ]:
sq.pl.spatial_scatter(adata, color='leiden', shape=None, dpi=200, size=24)

In [ ]:
tmp = adata[(adata.obs['leiden']=='3') |
            (adata.obs['leiden']=='0')]

In [ ]:

sc.tl.pca(tmp, svd_solver='arpack')
sc.pp.neighbors(tmp, n_neighbors=20, n_pcs=50)
sc.tl.umap(tmp)
sc.tl.leiden(tmp)
sc.pl.umap(tmp, color='leiden')

In [ ]:
sq.pl.spatial_scatter(tmp, color='leiden', shape=None, dpi=200, size=24)

In [ ]:
sc.pl.spatial(adata, 
              size=2,
              spot_size=300,
              color=['tbk1','ptpro','tcap', 'mt2'])

In [ ]:
gmm1 = fit_gmm(adata, 'mt2')
gmm2 = fit_gmm(adata, 'ptpro')

In [ ]:
distribution_distance(gmm1, gmm2)

In [ ]:
view_gmm(gmm1)

In [ ]:
view_gmm(gmm2, '2d')